# Data Exploration for Client Recommendation

Before doing client recommendation, I'm going to do some data analysis to see whether my plan is practical.

In [40]:
import pandas as pd
import numpy as np
import math

In [5]:
all_order_train = pd.read_pickle('../all_order_train.pkl')
all_order_test = pd.read_pickle('../all_order_test.pkl')
print(all_order_train.shape, all_order_test.shape)

all_order_train.head()

(33720820, 12) (98286, 12)


,order_id,user_id,order_number,order_hour_of_day,product_id,purchase_date,merchant,product_name,price,aisle,department,week_number
0,2539329,1,1,8,196,2019-02-13,L&M Markets (Hometown Grocers Co-op),Soda,3.08,soft drinks,beverages,7
1,2231262,31,17,11,196,2019-03-10,L&M Markets (Hometown Grocers Co-op),Soda,3.08,soft drinks,beverages,10
2,3058369,195,34,10,196,2019-02-25,L&M Markets (Hometown Grocers Co-op),Soda,3.08,soft drinks,beverages,9
3,2257155,951,3,20,196,2019-03-09,L&M Markets (Hometown Grocers Co-op),Soda,3.08,soft drinks,beverages,10
4,1121647,992,7,10,196,2019-03-08,L&M Markets (Hometown Grocers Co-op),Soda,3.08,soft drinks,beverages,10


## Shared Functions

In [18]:
def get_percentile(col):
    result = {'min': np.percentile(col, 0), '1%':np.percentile(col, 1),
             '5%':np.percentile(col, 5), '15%':np.percentile(col, 15),
             '25%':np.percentile(col, 25), '50%':np.percentile(col, 50), '75%':np.percentile(col, 75),
             '85%':np.percentile(col, 85), '95%':np.percentile(col, 95), '99%':np.percentile(col, 99),
              'max':np.percentile(col, 100)}
    return result

## Client Appearance Distribution

In [10]:
client_weeks_df = all_order_train[['user_id', 'week_number']].astype('str').drop_duplicates()\
          .groupby(['user_id'], as_index=False)['week_number']\
          .agg(['count']).reset_index()\
          .sort_values(['count'], ascending=False)

client_weeks_df.head()

,user_id,count
126573,28325,6
135599,36449,6
172385,69557,6
81882,173692,6
9358,10842,6


In [12]:
client_weeks_df['count'].value_counts()

5    61256
4    61174
3    51166
2    20929
6    10836
1      848
Name: count, dtype: int64

## Client Purchase Ability to Merchant Avg Clients

* `merchant_match_score = abs(1 - target_client_avg_purchase_dollars / merchannt_avg_sales_per_client)`
  * This score will be used to find merchant candidates in collaborative filtering
  * Smaller the score, the better match

In [16]:
merchant_sales_df = all_order_train[['merchant', 'user_id', 'price']].drop_duplicates()\
          .groupby(['merchant'], as_index=False)\
          .agg({'user_id': 'count', 'price': 'sum'})

merchant_sales_df.head()

,merchant,user_id,price
0,49th Parallel Grocery,1211766,1.452954e+07
1,Asian Food Centre,1210187,1.451086e+07
2,Askew's Foods,1203583,1.443146e+07
3,Avril (Health Supermarket),1200036,1.439485e+07
4,Calgary Co-op,1017673,1.218554e+07


In [17]:
merchant_sales_df['avg_sales_per_client'] = round(merchant_sales_df['price']/merchant_sales_df['user_id'], 4)

merchant_sales_df.head()

,merchant,user_id,price,avg_sales_per_client
0,49th Parallel Grocery,1211766,1.452954e+07,11.9904
1,Asian Food Centre,1210187,1.451086e+07,11.9906
2,Askew's Foods,1203583,1.443146e+07,11.9904
3,Avril (Health Supermarket),1200036,1.439485e+07,11.9953
4,Calgary Co-op,1017673,1.218554e+07,11.9739


In [19]:
get_percentile(merchant_sales_df['avg_sales_per_client'])

{'min': 11.8612,
 '1%': 11.87368,
 '5%': 11.907,
 '15%': 11.94756,
 '25%': 11.9562,
 '50%': 11.9693,
 '75%': 11.9822,
 '85%': 11.986419999999999,
 '95%': 11.99046,
 '99%': 11.993044000000001,
 'max': 11.9953}

* Seems all the merchants have similar average sales per client... Since I was using uniform random distribution...

In [20]:
client_purchase_df = all_order_train[['user_id', 'price', 'order_id']].drop_duplicates()\
          .groupby(['user_id'], as_index=False)\
          .agg({'price': 'sum', 'order_id': 'count'})

client_purchase_df.head()

,user_id,price,order_id
0,1,560.43,70
1,2,2685.71,226
2,3,994.48,88
3,4,282.01,18
4,5,543.37,46


In [21]:
client_purchase_df['avg_purchase'] = round(client_purchase_df['price']/client_purchase_df['order_id'], 4)

client_purchase_df.head()

,user_id,price,order_id,avg_purchase
0,1,560.43,70,8.0061
1,2,2685.71,226,11.8837
2,3,994.48,88,11.3009
3,4,282.01,18,15.6672
4,5,543.37,46,11.8124


In [22]:
get_percentile(client_purchase_df['avg_purchase'])

{'min': 2.01,
 '1%': 5.964432,
 '5%': 7.968640000000001,
 '15%': 9.576039999999999,
 '25%': 10.4277,
 '50%': 11.8533,
 '75%': 13.2933,
 '85%': 14.14598,
 '95%': 15.9362,
 '99%': 19.205927999999982,
 'max': 54.91}

* So the `merchant_match_score` distribution will be similar to this.

## Merchant Weekly Discounts

In [25]:
merchant_weekly_df = all_order_train[['merchant', 'week_number', 'price']].drop_duplicates()\
          .groupby(['merchant', 'week_number'], as_index=False)['price']\
          .agg(['sum']).reset_index()

merchant_weekly_df.head()

,merchant,week_number,sum
0,49th Parallel Grocery,7,61694.99
1,49th Parallel Grocery,8,71754.08
2,49th Parallel Grocery,9,72837.60
3,49th Parallel Grocery,10,70194.54
4,49th Parallel Grocery,11,69998.58


In [28]:
merchant_weekly_df['discount'] = merchant_weekly_df['sum']*0.000005
merchant_weekly_df.head()

,merchant,week_number,sum,discount
0,49th Parallel Grocery,7,61694.99,0.308475
1,49th Parallel Grocery,8,71754.08,0.358770
2,49th Parallel Grocery,9,72837.60,0.364188
3,49th Parallel Grocery,10,70194.54,0.350973
4,49th Parallel Grocery,11,69998.58,0.349993


In [29]:
get_percentile(merchant_weekly_df['discount'])

{'min': 0.0025241,
 '1%': 0.008776575000000002,
 '5%': 0.03578682,
 '15%': 0.09620614499999992,
 '25%': 0.1385151,
 '50%': 0.22229840000000023,
 '75%': 0.294991625,
 '85%': 0.32146624500000054,
 '95%': 0.3490226499999998,
 '99%': 0.35961323399999984,
 'max': 0.3641880000000004}

## Product Similarity

In [30]:
merchant_prod_df = pd.read_pickle('../merchant_product.pkl')
print(merchant_prod_df.shape)

merchant_prod_df.head()

(1439026, 8)


,merchant,product_id,product_name,aisle_id,department_id,aisle,department,price
0,L&M Markets (Hometown Grocers Co-op),196,Soda,77,7,soft drinks,beverages,3.08
1,Fairway Markets,196,Soda,77,7,soft drinks,beverages,3.08
2,Coleman's,196,Soda,77,7,soft drinks,beverages,3.08
3,Saskatoon Co-op,196,Soda,77,7,soft drinks,beverages,3.08
4,Asian Food Centre,196,Soda,77,7,soft drinks,beverages,3.08


In [31]:
prod_category = merchant_prod_df[['aisle_id', 'department_id', 'aisle', 'department']].drop_duplicates()
prod_category = prod_category.sort_values(by=['aisle_id', 'department_id'])
print(prod_category.shape)

prod_category.head()

(134, 4)


,aisle_id,department_id,aisle,department
807372,1,20,prepared soups salads,deli
248856,2,16,specialty cheeses,dairy eggs
379563,3,19,energy granola bars,snacks
1268575,4,9,instant foods,dry goods pasta
755893,5,13,marinades meat preparation,pantry


In [33]:
prod_category = prod_category.sort_values(by=['department_id', 'aisle_id'])
print(prod_category.shape)

prod_category.head()

(134, 4)


,aisle_id,department_id,aisle,department
938953,34,1,frozen meat seafood,frozen
866582,37,1,ice cream ice,frozen
827740,38,1,frozen meals,frozen
856850,42,1,frozen vegan vegetarian,frozen
902380,52,1,frozen breakfast,frozen


In [34]:
prod_df = merchant_prod_df[['product_id', 'product_name', 'aisle_id', 'department_id', 'aisle', 'department']].drop_duplicates()
print(prod_df.shape)

prod_df.head()

(49685, 6)


,product_id,product_name,aisle_id,department_id,aisle,department
0,196,Soda,77,7,soft drinks,beverages
49,46149,Zero Calorie Cola,77,7,soft drinks,beverages
98,17769,Diet Caffeine Free Naturally Flavored Lemon Li...,77,7,soft drinks,beverages
147,21108,Cherry Vanilla Creme Soda,77,7,soft drinks,beverages
196,39409,"Ginger Root Beer, Naturally Flavored Zero Calo...",77,7,soft drinks,beverages


In [38]:
prod_df['norm_aisle'] = round((prod_df['aisle_id'] - min(prod_df['aisle_id']))/(max(prod_df['aisle_id']) - min(prod_df['aisle_id'])), 4)
prod_df['norm_dept'] = round((prod_df['department_id'] - min(prod_df['department_id']))/(max(prod_df['department_id']) - min(prod_df['department_id'])), 4)

print(min(prod_df['norm_aisle']), max(prod_df['norm_aisle']), min(prod_df['norm_dept']), max(prod_df['norm_dept']))

prod_df.head()

0.0 1.0 0.0 1.0


,product_id,product_name,aisle_id,department_id,aisle,department,norm_aisle,norm_dept
0,196,Soda,77,7,soft drinks,beverages,0.5714,0.3
49,46149,Zero Calorie Cola,77,7,soft drinks,beverages,0.5714,0.3
98,17769,Diet Caffeine Free Naturally Flavored Lemon Li...,77,7,soft drinks,beverages,0.5714,0.3
147,21108,Cherry Vanilla Creme Soda,77,7,soft drinks,beverages,0.5714,0.3
196,39409,"Ginger Root Beer, Naturally Flavored Zero Calo...",77,7,soft drinks,beverages,0.5714,0.3


In [39]:
target_prod = 196

other_prod_df = prod_df.loc[prod_df['product_id'] != target_prod]
other_prod_df.head()

,product_id,product_name,aisle_id,department_id,aisle,department,norm_aisle,norm_dept
49,46149,Zero Calorie Cola,77,7,soft drinks,beverages,0.5714,0.3
98,17769,Diet Caffeine Free Naturally Flavored Lemon Li...,77,7,soft drinks,beverages,0.5714,0.3
147,21108,Cherry Vanilla Creme Soda,77,7,soft drinks,beverages,0.5714,0.3
196,39409,"Ginger Root Beer, Naturally Flavored Zero Calo...",77,7,soft drinks,beverages,0.5714,0.3
245,21462,Zero Calorie Cola Soda,77,7,soft drinks,beverages,0.5714,0.3


In [41]:
def get_prod_similarity(target_aisle, target_dept, aisle, dept):
    similarity = 1 - math.sqrt((pow(target_aisle - aisle, 2) + (pow(target_dept - dept, 2)))/2)
    
    return round(similarity, 4)

In [49]:
import warnings
warnings.filterwarnings("ignore")

target_aisle = prod_df.loc[prod_df['product_id'] == target_prod]['norm_aisle'].values[0]
target_dept = prod_df.loc[prod_df['product_id'] == target_prod]['norm_dept'].values[0]

other_prod_df['similarity'] = other_prod_df.apply(lambda r: get_prod_similarity(target_aisle, target_dept,
                                                                                r['norm_aisle'], r['norm_dept']), axis=1)
other_prod_df.head()

,product_id,product_name,aisle_id,department_id,aisle,department,norm_aisle,norm_dept,similarity
49,46149,Zero Calorie Cola,77,7,soft drinks,beverages,0.5714,0.3,1.0
98,17769,Diet Caffeine Free Naturally Flavored Lemon Li...,77,7,soft drinks,beverages,0.5714,0.3,1.0
147,21108,Cherry Vanilla Creme Soda,77,7,soft drinks,beverages,0.5714,0.3,1.0
196,39409,"Ginger Root Beer, Naturally Flavored Zero Calo...",77,7,soft drinks,beverages,0.5714,0.3,1.0
245,21462,Zero Calorie Cola Soda,77,7,soft drinks,beverages,0.5714,0.3,1.0


In [50]:
get_percentile(other_prod_df['similarity'])

{'min': 0.388,
 '1%': 0.4214,
 '5%': 0.4901,
 '15%': 0.5521,
 '25%': 0.6029,
 '50%': 0.6942,
 '75%': 0.7627,
 '85%': 0.7933199999999546,
 '95%': 0.9096,
 '99%': 0.9642,
 'max': 1.0}